# Lab 6

Welcome to lab 6!  In lab 5, we used simulation to investigate the random variation in an estimate that was based on a random sample.  Now we'll flip that idea on its head to make it useful for *statistical inference*.

# 0. Preliminaries

As usual, **run the cell below** to prepare the lab and the automatic tests.

In [1]:
# Run this cell, but please don't change it.

# These lines import the NumPy and datascience modules.
import numpy as np
# This way of importing the datascience module lets you write "Table" instead
# of "datascience.Table".  The "*" means "import everything in the module."
from datascience import *

# These lines set up visualizations.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.assignment import load_assignment 
lab06 = load_assignment('longlab06.ok')

# 1. Warplanes again

Last time, we saw how various estimates of the number of warplanes would typically vary.  Now let's make that more useful by producing *confidence intervals* to quantify our uncertainty in a *given estimate*.

Remember the setup: We (the RAF in World War II) want to know the number of warplanes fielded by the Germans.  That's equal to the largest serial number on any of the warplanes.  We only see a small number of serial numbers (assumed to be a random sample from among all the serial numbers), so we have to use estimation.

To simulate this, we're going to hide the true number (`N`) from you.  You'll have access only to this random sample:

In [7]:
#FIXME: Delete
# The biggest serial number.
N = 150
# The number of observations.
num_observations = 17

def simulate_observations():
    return np.random.randint(1, N+1, num_observations)

t = Table().with_column("serial number", simulate_observations())
t.to_df().to_csv("serial_numbers.csv", index=False)

In [10]:
observations = Table.read_table("serial_numbers.csv").column(0)
num_observations = len(observations)

def plot_serial_numbers(numbers):
    Table().with_column("serial number", numbers).hist(bins=np.arange(1, 200))
    plt.ylim(0, .25)

plot_serial_numbers(observations)

Your job is to estimate `N`.  You'll see that a confidence interval will help you understand how sure you should be about your answer.

We saw that one way to estimate `N` was to take twice the mean of the serial numbers we see.

In [12]:
def mean_based_estimator(nums):
    return 2*np.average(nums)

mean_based_estimate = mean_based_estimator(observations)

**Question 1.1.** In this particular sample, what's the biggest number?  Compute it, giving it the name `max_estimate`. Think about what that implies about `mean_based_estimate`.

In [14]:
max_estimate = ...
max_estimate

In [13]:
_ = lab06.grade("q11")

`N` is surely at least as big as the biggest serial number in our sample.  So in this case, we can tell that the mean-based estimate is off.

If we knew the sampling distribution of the mean-based estimate, we'd know how far off it typically is.  Unfortunately, since we don't know `N`, we can't just simulate to compute that sampling distribution.  Remember, our `simulate_observations` function in lab 5 looked like this:

In [ ]:
# We can't write this part, because we don't know N!
N = ...

def simulate_observations():
    # You'll get an error message if you try to call this
    # function.
    return np.random.randint(1, N+1, num_observations)

## 1.1. Resampling
Instead, we'll use resampling.  That is, instead of sampling from the true distribution of serial numbers, we'll sample from our sample.

In [66]:
observations_table = Table().with_column("serial number", observations)

def simulate_resample():
    return observations_table.sample(num_observations, with_replacement=True).column(0)

# This is a little magic to make sure that you see the same results
# we did.
np.random.seed(123)
one_resample = simulate_resample()

**Question 1.1.1.** Make a histogram to display the distribution of serial numbers in `one_resample`.  Use the function `plot_serial_numbers`, which was defined and used a few screens above.

In [67]:
plot_serial_numbers(one_resample)

To compare, here's the actual observations again:

In [68]:
plot_serial_numbers(observations)

You should see that the resample has only the elements of the sample.  Some are repeated several times, and some don't get into the resample at all.

The mean of the resample is:

In [69]:
mean_based_estimator(one_resample)

Let's repeat this many times and see what we get.

In [49]:
num_simulations = 20000

#FIXME: Put in library
def repeat(f, repetitions):
    return [f() for i in range(repetitions)]

def simulate_estimates(estimator, simulator):
    simulations = Table().with_column("resample", repeat(simulator, num_simulations))
    return simulations.apply(estimator, "resample")

bins = np.arange(50, 250, 1)

def draw_simulated_distribution(estimator, simulator):
    Table().with_column("estimates", simulate_estimates(estimator, simulator)).hist("estimates", bins=bins)

draw_simulated_distribution(mean_based_estimator, simulate_resample)

We call this a "resampling" or "bootstrap" distribution of our estimate.

Its interpretation is: If the population looked like our sample, then we'd expect our estimator usually to produce estimates between around 80 and 170, and often between around 100 and 140.  We just looked at the histogram to come up with those numbers.

We can be more quantitative about this by computing *percentiles* of the resampling distribution.

In [50]:
resample_estimates = simulate_estimates(mean_based_estimator, simulate_resample)

def coverage_interval(numbers, coverage):
    return np.percentile(numbers, [(100-coverage)/2, (100+coverage)/2])

def print_coverage_interval(numbers, coverage):
    interval = coverage_interval(numbers, coverage)
    pattern = "If the population looked like our sample, our sample-based estimates of N would be between {:.2f} and {:.2f} {:.1f}% of the time."
    message = pattern.format(interval.item(0), interval.item(1), coverage)
    print(message)

print_coverage_interval(resample_estimates, 95)
print_coverage_interval(resample_estimates, 99.9)

Below we've written some code to display a resampling distribution with these percentiles overlaid.

In [76]:
def draw_distribution_and_interval(estimator, simulator, coverage):
    estimates = simulate_estimates(estimator, simulator)
    interval = coverage_interval(estimates, coverage)
    Table().with_column("estimates", estimates).hist("estimates", bins=bins)
    
    def draw_bar(x):
        plt.plot([x, x], [0, .04], color="red")
    
    draw_bar(interval.item(0))
    draw_bar(interval.item(1))
    plt.title("{:.2f}% coverage interval".format(coverage))

draw_distribution_and_interval(mean_based_estimator, simulate_resample, 95)
draw_distribution_and_interval(mean_based_estimator, simulate_resample, 99.9)

## 1.2. Confidence Intervals
Now comes the tricky part.  We'd like to move from a statement like this:

> "If the population looked like this, then we'd usually get estimates for `N` between A and B."

...to what we really want:

> "We claim `N` is actually between X and Y, and usually our claims are right."

We can't cover the intricacies of the idea in full here.  But the idea is to flip our thinking around.  Assume that the *variability* in estimates would look roughly the same for all the plausible values of `N`.  Then if we put an interval around our estimate of `N` that covers 95% of the resamples, it's also true that 95% of the time our estimate will be close enough to `N` that our interval will cover `N`.

Anyway, here is the method:

In [77]:
Table().with_columns(["resampling estimates", resample_estimates, "estimates (if N were 140)", resample_estimates - np.mean(resample_estimates) + 140]).hist(bins=bins)

Then 95% of the time, our estimates would still cover 121.8, which is the mean in our actual sample.

In [ ]:
def make_sample_simulator(simulated_N):
    def simulator():
        return np.random.randint(1, simulated_N+1, num_observations)
    return simulator

draw_simulated_distribution(mean_based_estimator, make_sample_simulator(140))

In [85]:
error = N - max_estimate
print("Our estimate is {} off from N.".format(error))

But maybe we got lucky or unlucky.  If we're deciding whether to use this method for a similar task, we'd like to know how it *typically* performs, not how it happened to perform this time.

To see what typically happens, we just run our simulation many times.  The cell below does that.

In [86]:
num_simulations = 20000
bins = np.arange(N - 60 + 1, N + 60 + 1, 1)

#FIXME: This should be in a library.
def repeat(f, repetitions):
    return [f() for i in range(repetitions)]

def draw_max_distribution():
    simulations = Table().with_column("observations", repeat(simulate_observations, num_simulations))
    maxes = simulations.apply(max, "observations")
    simulations.with_column("max", maxes).hist("max", bins=bins)

draw_max_distribution()

This histogram says how often we get different estimates of `N` using this technique.  It's a histogram displaying the *probability distribution* of this estimate.  Sometimes that's called, somewhat confusingly, the *sampling distribution* of the estimate.

The first thing to notice is that we don't always get the right answer.  You might say that we usually get close to the right answer, but that depends on your definition of "close."  When you're trying to figure out how many warplanes the Germans have, you might want to be even more accurate!

You should also find that the sample max is never bigger than `N`, but it's sometimes smaller.  In other words, you only ever *underestimate* `N` using this technique.

**Question 1.2.2.** Try to explain this phenomenon in your own words.  Discuss with your neighbor if you're stuck!

*Write your answer here, replacing this text.  (Double-click this cell to edit it, and click the "run cell" button to switch back to display mode.)*

##### Extrapolating from the average
Here's another idea that comes from looking at the probability histogram of the serial numbers.

Since the serial numbers are evenly distributed from 1 to `N`, the average of all the serial numbers is roughly in the middle: $\frac{\texttt{N}}{2}$.  Further, the law of averages says (heuristically) that the average of a sample is likely to be close to the average of the population it was sampled from.  So if we multiply the average of our observations by 2, we might get something close to `N`.

If you prefer symbols, here's another way to say that:

$$\texttt{average}(\texttt{all serial numbers}) \approx \frac{\texttt{N}}{2} \\
  \overset{\text{by law of averages}}{\implies} \texttt{average}(\texttt{observations}) \approx \frac{\texttt{N}}{2} \\
  \implies 2 \times \texttt{average}(\texttt{observations}) \approx \texttt{N}$$

Here's a function that computes twice the mean of an array of serial numbers:

In [87]:
def mean_based_estimator(nums):
    return 2*np.average(nums)

**Question 1.2.3.** Use that function to estimate `N` from `observations`.  Call the result `mean_based_estimate`.

In [88]:
# Compute mean_based_estimate using your simulated observations.
mean_based_estimate = ...
mean_based_estimate

In [89]:
_ = lab05.grade('q123')

Again, it's not clear whether this estimate was about as accurate we'd expect from this method, or just a fluke.  As before, we can see how this estimator works by simulating.

In [90]:
def draw_twice_mean_distribution():
    simulations = Table().with_column("observations", repeat(simulate_observations, num_simulations))
    twice_means = simulations.apply(mean_based_estimator, "observations")
    simulations.with_column("twice means", twice_means).hist("twice means", bins=bins)

draw_twice_mean_distribution()

**Question 1.2.4.** Among the two estimators we've seen so far, which would you prefer?  Think about the criteria you would use to decide this.

*Write your answer here, replacing this text.*

Notice something about the histograms we've seen so far:

1. The twice-the-mean estimator has (roughly) a Normal distribution.
2. The max estimator doesn't have a Normal distribution.  Classical statistical techniques, which assume sampling distributions are roughly Normal, wouldn't work very well to help us understand how that estimator works.

##### Something more clever
So far, it looks like our choice is between an estimator that chronically underestimates our target and one that tends to make large errors.  Let's take a look at the distribution of maxes again:

In [91]:
draw_max_distribution()

A good question to ask about this histogram is: "Why can't we just shift the estimates over a bit to get more of them close to 120?"

Another way of thinking about this is that it's unlikely we actually get the biggest serial number in our sample, so it makes sense to guess that the real `N` is something a bit higher than the biggest serial number we observe.  For example, if we see 110, it's probably safe to guess that `N` is 111.

Well, we can act on that.  Let's try a few different "shifted" estimators -- the max of the sample, plus a bit.  First we'll define some functions to make that easier.  (If you can't follow these, that's okay; there will be a lab session later where you can learn how to use functions like this.)

In [92]:
def max_plus_n(n):
    def max_func(nums):
        return max(nums) + n
    return max_func

def simulate_estimates(estimator):
    simulations = Table().with_column("observations", repeat(simulate_observations, num_simulations))
    return simulations.apply(estimator, "observations")

def draw_sampling_distribution(estimator):
    Table().with_column("estimates", simulate_estimates(estimator)).hist("estimates", bins=bins)

draw_sampling_distribution(max_plus_n(5))
draw_sampling_distribution(max_plus_n(10))
draw_sampling_distribution(max_plus_n(15))

In the first two, more of the weight of the bars is near 120.  That means we're typically getting closer to 120 this way, which is what we want.  15 goes too far.  So it looks like shifting over by 5 or 10 is a good idea.

Why?  Here's an idea pointed out by a student in Data 8 lecture.  Think about what would happen if our samples came out evenly-spaced in the interval 1 to 120.  The space between them would be $\frac{120}{14}$, or roughly 8.5.  They'd look like this:

In [93]:
plot_serial_numbers(np.arange(N/num_observations, N, N/num_observations))

The biggest observation, hence our max estimate, would be around 8.5 less than 120.  So we can correct for this by adding back in $\frac{120}{14}$.

In [96]:
adjusted_estimator = max_plus_n(N/num_observations)
draw_sampling_distribution(adjusted_estimator)

It turns out that this is a pretty good way to estimate the number of warplanes.

It's worth noting again that the sampling distribution doesn't look very Normal.  Classical statistical methods wouldn't help you understand it, but we can simulate it precisely with a few lines of code.